In [2]:
import pandas as pd
import numpy as np
import os

#locating all the two data and read the csv file
school_path = os.path.join(".","Resources","schools_complete.csv")
students_path = os.path.join(".","Resources","students_complete.csv")

df_school = pd.read_csv(school_path)
df_students = pd.read_csv(students_path)

In [3]:
# merge two csv file together into one
df_complete = pd.merge(df_students, df_school, how="left", on=["school_name", "school_name"])

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [4]:
# calculation for the summary of local gov area 
num_school = len(df_school)
num_student = len(df_students)
budget = df_school["budget"].sum()
ave_maths = df_complete["maths_score"].mean()
ave_read = df_complete["reading_score"].mean()
# conditional counts with len(df[df["column"] == value])
pass_maths = len(df_complete[df_complete["maths_score"] >= 50])/num_student
pass_read = len(df_complete[df_complete["reading_score"] >= 50])/num_student
# conditional counts with two different condition len(df[(df["column1"] == value1) & (df["column2"] == value2)])
overall_pass = len(df_complete[(df_complete["maths_score"] >= 50) & (df_complete["reading_score"] >= 50)])/num_student

In [5]:
# list of the title and numbers in dataframe
title_area = ["Total Schools",
         "Total Students",
         "Total Budget",
         "Average Maths Score",
         "Average Reading Score",
         "% Passing Maths",
         "% Passing Reading",
         "% Overall Passing"
        ]
# list of the first row, need to [[row1],[row2],[row3]]
# formatting each cell thousands separators f"{:,}", 2 decimal places f"{:,}", f"{:2%}"
results_area = [[num_school,
           f"{num_student:,}",
           f"${budget:,}",
           f"{ave_maths:.2f}",
           f"{ave_read:.2f}",
           f"{pass_maths:.2%}",
           f"{pass_read:.2%}",
           f"{overall_pass:.2%}"
           ]]

<font size= "7">__Local Government Area Summary__</font>

In [6]:
area_summary = pd.DataFrame(data = results_area, columns = title_area, index = [0])
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34,69.98,86.08%,84.43%,72.81%


In [7]:
# Grouping different schools
grp_school = df_complete.groupby(["school_name"])

In [30]:
# arranging values into the school group
s_type = grp_school["type"].first()
s_budget = grp_school["budget"].first()

# number of students in each school using .count()
s_student = grp_school["Student ID"].count()
# average of student budget simply using s_budget/s_student
s_ave_budget = s_budget/s_student
# average of scores using .mean()
s_ave_maths = grp_school["maths_score"].mean()
s_ave_read = grp_school["reading_score"].mean()

# identifying the number of student passing the maths and reading scores
# using lambda x: (x >= 50) giving true/false results and then sum up all the true(or all the one)
# create a new list to calculate the number of student pass/ total student; 
# the index is grp_school.groups (which are school name) becasue the index after groupby is the school names
s_pass_maths_num = grp_school["maths_score"].apply(lambda x: (x >= 50).sum())
s_pass_maths_per = pd.Series(s_pass_maths_num.values/s_student.values, index = grp_school.groups)

# another method to apply for conditions, this time, the variable x is df
# the double [[""]] means df[[a list of column]], so the second [] means a list of columns
# therefore, in lambda function, column name need to be included, also means (df.reading_score >= 50)
s_pass_read_num = grp_school[["reading_score"]].apply(lambda x: (x.reading_score >= 50).sum())
s_pass_read_per = pd.Series(s_pass_read_num.values/s_student.values, index = grp_school.groups)

# using the same method as above but with two conditions
s_pass_over_num = grp_school[["reading_score", "maths_score"]].apply(lambda x : ((x.reading_score >= 50) & (x.maths_score >= 50)).sum())
# getting the values from s_pass_over_num and divde the number of students, (.values is used to extract numbers)
# create a new series with pd.Series and the index is grp_school (using .groups to identify the group name)
s_pass_over_per = pd.Series(s_pass_over_num.values/s_student.values, index = grp_school.groups)

In [31]:
# grouping all the reults into each column using pd.concat([xxx], axis = "column/row")
per_school_summary = pd.concat([s_type, 
                       s_student, 
                       s_budget,
                       s_ave_budget,
                       s_ave_maths,
                       s_ave_read,
                       s_pass_maths_per,
                       s_pass_read_per,
                       s_pass_over_per
                      ], axis = "columns", sort=False)
# renaming all the column name
per_school_summary = per_school_summary.rename(columns={"type":"School Type",
                                                        "Student ID":"Total Students",
                                                        "budget":"Total School Budget",
                                                        0:"Per Student Budget",
                                                        "maths_score":"Average Maths Score",
                                                        "reading_score":"Average Reading Score",
                                                        1:"% Passing Maths",
                                                        2:"% Passing Reading",
                                                        3:"% Overall Passing"
                                                        })
# create a new table just .style.format, using xxx = xxx.style.format usually give a more stable result than inpalce
per_school_summary_formated = per_school_summary.style.format({"Total School Budget": "${:,}",
                                                    "Per Student Budget": "${:.2f}",
                                                    "Average Maths Score": "{:.2f}",
                                                    "Average Reading Score": "{:.2f}",
                                                    "% Passing Maths": "{:.2%}",
                                                    "% Passing Reading": "{:.2%}",
                                                    "% Overall Passing": "{:.2%}"
                                                    })

<font size= "7">__School Summary__</font>

In [10]:
per_school_summary_formated

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,1858,"$1,081,356",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,2949,"$1,884,411",$639.00,68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,2739,"$1,763,916",$644.00,69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,1468,"$917,500",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,4635,"$3,022,020",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087",$581.00,72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,2917,"$1,910,635",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,4761,"$3,094,650",$650.00,68.84,69.04,82.06%,81.98%,67.19%
Pena High School,Independent,962,"$585,858",$609.00,72.09,71.61,91.68%,86.59%,79.21%


<font size= "6">__Top Performing Schools (By % Overall Passing)__</font>

In [11]:
# sorting the table by overall passing rate
# sort_values(by = "which column want to be sorted", ascending = True/False) 
# ascending = True (sorted from smallest) False (sorted from largest)
top_school = per_school_summary.sort_values(by = "% Overall Passing", ascending = False)
top_school = top_school.head()
top_school_format = top_school.style.format({"Total School Budget": "${:,}",
                                                    "Per Student Budget": "${:.2f}",
                                                    "Average Maths Score": "{:.2f}",
                                                    "Average Reading Score": "{:.2f}",
                                                    "% Passing Maths": "{:.2%}",
                                                    "% Passing Reading": "{:.2%}",
                                                    "% Overall Passing": "{:.2%}"
                                                    })
top_school_format

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Cabrera High School,Independent,1858,"$1,081,356",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Bailey High School,Government,4976,"$3,124,928",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Wright High School,Independent,1800,"$1,049,400",$583.00,72.05,70.97,91.78%,86.67%,79.72%
Rodriguez High School,Government,3999,"$2,547,363",$637.00,72.05,70.94,90.80%,87.40%,79.42%


<font size= "6">__Bottom Performing Schools (By % Overall Passing)__</font>

In [33]:
bottom_schools = per_school_summary.sort_values(by = "% Overall Passing", ascending = True)
bottom_schools = bottom_schools.head()
bottom_schools_format = bottom_schools.style.format({"Total School Budget": "${:,}",
                                                    "Per Student Budget": "${:.2f}",
                                                    "Average Maths Score": "{:.2f}",
                                                    "Average Reading Score": "{:.2f}",
                                                    "% Passing Maths": "{:.2%}",
                                                    "% Passing Reading": "{:.2%}",
                                                    "% Overall Passing": "{:.2%}"
                                                    })
bottom_schools_format

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Huang High School,Government,2917,"$1,910,635",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,4761,"$3,094,650",$650.00,68.84,69.04,82.06%,81.98%,67.19%
Wilson High School,Independent,2283,"$1,319,574",$578.00,69.17,68.88,82.79%,81.30%,67.46%
Ford High School,Government,2739,"$1,763,916",$644.00,69.09,69.57,82.44%,82.22%,67.47%


<font size= "6">__Maths Scores by Year__</font>

In [13]:
# need two groups results, so need to group two group together using groupby
grp_year_school = df_complete.groupby(["school_name", "year"])
# calucate the mean math results from the two group, will output a stack table (print math_mean to see the result)
math_mean = grp_year_school["maths_score"].mean()
# reorganise the year to header using unstack()
maths_scores_by_year = math_mean.unstack()
maths_scores_by_year.index.name=None
maths_scores_by_year.columns.name= None
maths_scores_by_year = maths_scores_by_year.rename(columns={9: "Year 9",
                                            10: "Year 10",
                                            11: "Year 11",
                                            12: "Year 12"
                                            })
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


<font size= "6">__Reading Score by Year__</font>

In [14]:
# calucate the mean reading results from the two group, will output a stack table (print math_mean to see the result)
read_mean = grp_year_school["reading_score"].mean()
# reorganise the year to header using unstack()
reading_scores_by_year = read_mean.unstack()
reading_scores_by_year.columns.name= None
reading_scores_by_year.index.name= None
reading_scores_by_year = reading_scores_by_year.rename(columns={9: "Year 9",
                                            10: "Year 10",
                                            11: "Year 11",
                                            12: "Year 12"
                                            })
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


<font size= "6">__Scores by School Spending__</font>

In [32]:
bins = [0, 585, 630, 645, 680]
grp_name = ["< $585", "$585 - 630", "$630 - 645", "$645 - 680"]
# getting the column that needed from the school summary table
spending_summary = per_school_summary.loc[:,["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
# adding a new column and seperate in numerous order using cut
# the budget data extracting from school summary table and bin = the start number for each group
# labels is the name for each group
# includes_lowest = True is including the bins number in the list, in this case, is false because that is the highest number for each grp
spending_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], bins, labels= grp_name, include_lowest= False)
# grouping the identified group together and avg value for the whole table 
spending_summary = spending_summary.groupby("Spending Ranges (Per Student)").mean()
spending_summary_formated = spending_summary.style.format({"Average Maths Score": "{:.2f}",
                                                    "Average Reading Score": "{:.2f}",
                                                    "% Passing Maths": "{:.2%}",
                                                    "% Passing Reading": "{:.2%}",
                                                    "% Overall Passing": "{:.2%}"
                                                    })
spending_summary_formated

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $585,71.36,70.72,88.84%,86.39%,76.72%
$585 - 630,72.07,71.03,91.52%,87.29%,79.88%
$630 - 645,69.85,69.84,84.69%,83.76%,71.00%
$645 - 680,68.88,69.05,81.57%,81.77%,66.76%


<font size = "6">__Scores by School Size__</font>

In [19]:
# same as the School spending table
bins = [0, 1000, 2000, 5000]
grp_name = ["Small (<1000)", "Medium (1000 - 2000)", "Large (2000 -5000)"]
size_summary =  per_school_summary.loc[:,["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
size_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins, labels= grp_name, include_lowest= False)
size_summary = size_summary.groupby("School Size").mean()
size_summary_formated = size_summary.style.format({"Average Maths Score": "{:.2f}",
                                                    "Average Reading Score": "{:.2f}",
                                                    "% Passing Maths": "{:.2%}",
                                                    "% Passing Reading": "{:.2%}",
                                                    "% Overall Passing": "{:.2%}"
                                                    })
size_summary_formated

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81%,87.56%,79.07%
Medium (1000 - 2000),71.42,70.72,89.85%,86.71%,78.04%
Large (2000 -5000),69.75,69.58,84.25%,83.30%,70.29%


<font size = "6">__Scores by School Type__</font>

In [21]:
type_summary = per_school_summary.loc[:,["School Type", "Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
# groupby school type and then avg
type_summary = type_summary.groupby("School Type").mean()
type_summary_formated = type_summary.style.format({"Average Maths Score": "{:.2f}",
                                            "Average Reading Score": "{:.2f}",
                                            "% Passing Maths": "{:.2%}",
                                            "% Passing Reading": "{:.2%}",
                                            "% Overall Passing": "{:.2%}"
                                            })
type_summary_formated

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46%,83.59%,70.70%
Independent,71.37,70.72,89.20%,86.25%,76.97%


<font size = "7">__Summary__</font>



 This study demonstrated the local school performance using their students performance in Mathematcis and reading examinations as an indicator. From all the 15 local schools in the area, the top 5 and bottom 5 schools are ranked by their students overall passing rate in Mathematics and reading examinations. The results suggested that government schools have a lower average score and overall passing rate compared to independent schools. Comparing to the schools with larger amount of students, the overall passing rate and scores are higher in the smaller schools.